# **BMB 306 Yapay Zekaya Giriş Dersi Dönem Sonu Projesi**

# Proje amacı: Resimdeki insanların sigara içip içmediğini söyler

Gürkan GÜR 2020717061

Mustafa KÖSE 2020717023

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import tensorflow as tf
from tensorflow import keras
from keras.applications.resnet import ResNet50
from keras.utils.vis_utils import plot_model
from glob import glob
from keras.models import Model
from keras.layers import Flatten, Dense
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator


In [4]:
# ResNet50 modelinin oluşturulması
Image_size = [224, 224]
valid_path = "/content/drive/MyDrive/BMB306_Donem_Sonu_Projesi/Smoker Detection/Testing"
train_path = "/content/drive/MyDrive/BMB306_Donem_Sonu_Projesi/Smoker Detection/Training"

resnet = ResNet50(include_top=False, input_shape=Image_size+[3], weights="imagenet")
plot_model(resnet)

for layer in resnet.layers:
    layer.trainable = False

94765736/94765736 [==============================] - 1s 0us/step


In [5]:
# Sınıf etiketlerinin belirlenmesi
folders = glob("/content/drive/MyDrive/BMB306_Donem_Sonu_Projesi/Smoker Detection/Testing/*")

In [6]:
# Modelin tamamlanması
x = Flatten()(resnet.output)
prediction = Dense(len(folders), activation="softmax")(x)

model = Model(inputs=resnet.input, outputs=prediction)
plot_model(model)

model.compile(loss="categorical_crossentropy", optimizer=Adam(), metrics=["accuracy"])

In [7]:
# Veri artırma ve ön işleme için veri üreteçlerinin tanımlanması
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [8]:
# Eğitim ve test veri setlerinin yüklenmesi
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size=(224, 224),
                                                 batch_size=32,
                                                 class_mode="categorical")

test_set = test_datagen.flow_from_directory(valid_path,
                                            target_size=(224, 224),
                                            batch_size=32,
                                            class_mode="categorical")


Found 716 images belonging to 2 classes.
Found 224 images belonging to 2 classes.


In [9]:
# Modelin eğitilmesi
r = model.fit(training_set,
              validation_data=test_set,
              epochs=50,
              steps_per_epoch=len(training_set),
              validation_steps=len(test_set),
              verbose=1)


Epoch 1/50
23/23 [==============================] - 216s 9s/step - loss: 2.9298 - accuracy: 0.4860 - val_loss: 1.6852 - val_accuracy: 0.5089
Epoch 2/50
23/23 [==============================] - 12s 534ms/step - loss: 1.0892 - accuracy: 0.5265 - val_loss: 0.6765 - val_accuracy: 0.6116
Epoch 3/50
23/23 [==============================] - 12s 534ms/step - loss: 0.8307 - accuracy: 0.5838 - val_loss: 0.7552 - val_accuracy: 0.5446
Epoch 4/50
23/23 [==============================] - 12s 499ms/step - loss: 0.7632 - accuracy: 0.5950 - val_loss: 0.8645 - val_accuracy: 0.5402
Epoch 5/50
23/23 [==============================] - 12s 533ms/step - loss: 1.0705 - accuracy: 0.5712 - val_loss: 2.3789 - val_accuracy: 0.5134
Epoch 6/50
23/23 [==============================] - 12s 539ms/step - loss: 1.2699 - accuracy: 0.5335 - val_loss: 0.7435 - val_accuracy: 0.6205
Epoch 7/50
23/23 [==============================] - 12s 490ms/step - loss: 0.7053 - accuracy: 0.6634 - val_loss: 0.6760 - val_accuracy: 0.6473
E

In [10]:
# Modelin kaydedilmesi
model.save("model.h5")

In [11]:
#İlgili kütüphanelerin import edilmesi.
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [12]:
#Eğitilen modelin yüklenmesi ve test edilecek resmin dosya yolunun belirtilmesi.
model = load_model("model.h5")
#tırnak içerisine istediğiniz görselin adresini atarak test edebilirisiniz
img_path = "/content/drive/MyDrive/BMB306_Donem_Sonu_Projesi/Smoker Detection/Testing/smoking/smoking_0060.jpg"

#Test edilecek resmin yüklenmesi, boyutunun 224x224 piksel olarak ayarlanması, numpy dizisine dönüştürülmesi, normalizasyonu ve boyutunun genişletilmesi.
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = x/255.0
x = np.expand_dims(x, axis=0)

#Modelin kullanılarak tahmin yapılması ve tahmin sınıfının belirlenmesi.
preds = model.predict(x)
pred_class = np.argmax(preds, axis=1)[0]

#Tahmin sınıfına göre "Sigara içiyor" veya "Sigara içmiyor" sonucunun belirlenmesi ve ekrana yazdırılması
if pred_class == 1:
    prediction = "Sigara içiyor"
else:
    prediction = "Sigara içmiyor"

print("Tahmin: ", prediction)

1/1 [==============================] - 1s 1s/step
Tahmin:  Sigara içiyor
